In [1]:
import krakenex
import datetime
import calendar
import pandas as pd
import time

%matplotlib inline

In [2]:
# takes date and returns nix time
def date_nix(str_date):
    return calendar.timegm(str_date.timetuple())

# takes nix time and returns date
def date_str(nix_time):
    return datetime.datetime.fromtimestamp(nix_time).strftime('%m, %d, %Y')

#return formated request data
def req(start, end, ofs):
    req_data = {'type': 'all',
                'trades': 'true',
                'start': str(date_nix(start)),
                'end': str(date_nix(end)),
                'ofs': str(ofs)
                }
    return req_data

In [3]:
k = krakenex.API()
k.load_key('kraken.key')

In [5]:


depth = k.query_public('Depth', {'pair': 'XXBTZUSD', 'count': '100'})
asks = depth['result']['XXBTZUSD']['asks']
bids = depth['result']['XXBTZUSD']['bids']
#print(asks)

df_asks = pd.DataFrame(data=asks, columns=['price','volume','timestamp'])
df_asks[['price', 'volume']] = df_asks[['price', 'volume']].astype(float)   
df_asks[['timestamp']] = df_asks[['timestamp']].astype(int) 
df_asks = df_asks.sort_values('timestamp')
#df_asks.plot(x='timestamp', y='price')

df_bids = pd.DataFrame(data=bids, columns=['price','volume','timestamp'])
df_bids[['price', 'volume']] = df_bids[['price', 'volume']].astype(float)  
df_bids[['timestamp']] = df_bids[['timestamp']].astype(int) 
df_bids = df_bids.sort_values('timestamp')
#df_bids.plot(x='timestamp', y='price')

ax = df_asks.plot(x='timestamp', y='price', label='asks')
df_bids.plot(ax=ax, x='timestamp', y='price', label='bids')

pd.DataFrame(columns=['timestamp','bid','ask']).to_csv('spread.csv', mode='a')
df_spread = pd.read_csv('spread.csv')

while True:
#for i in range(5):
    spread = k.query_public('Spread', {'pair': 'XXBTZUSD', 'since': '1501500000'})

    df_temp = pd.DataFrame(data=spread['result']['XXBTZUSD'], columns=['timestamp','bid','ask'])
    df_temp[['bid', 'ask']] = df_temp[['bid', 'ask']].astype(float)  
    df_temp[['timestamp']] = df_temp[['timestamp']].astype(int)
    
    print(df_temp)
    time.sleep(5)

    df_spread = pd.concat([df_spread,df_temp]).drop_duplicates().reset_index(drop=True)
    df_spread = df_spread.sort_values('timestamp')
    
    df_spread.to_csv('spread.csv', mode='a')
    
print(df_spread)

df_spread.plot(x='timestamp', y=['ask','bid'])



#datetime.datetime.fromtimestamp(int("1284101485")).strftime('%Y-%m-%d %H:%M:%S')

gaierror: [Errno -2] Name or service not known